# Objective


## Demonstate via code how Docker and AWS ECR interacts to:
* Pushing Images
* Pulling Images
* Logging In
* Listing Repositories
* Deleting Images
* Managing Lifecycle Policies
* Integrating with CI/CD
* Scanning Images
* Copying Images


## 1. Push image
To push a Docker image to an AWS Elastic Container Registry (ECR) using Python, you can use the AWS SDK for Python, commonly known as Boto3. Before running the code, make sure you have the AWS CLI installed and configured with your AWS credentials.

Below is the Python script to push a Docker image to an AWS ECR repository:

In [ ]:
import boto3
import subprocess

# Set your AWS region and ECR repository name
aws_region = 'your-aws-region'
ecr_repository_name = 'your-ecr-repo-name'

# Create a Boto3 ECR client
ecr_client = boto3.client('ecr', region_name=aws_region)

# Get the ECR login token to authenticate Docker to your registry
ecr_login = ecr_client.get_authorization_token()
registry = ecr_login['authorizationData'][0]['proxyEndpoint']
username = 'AWS'
password = ecr_login['authorizationData'][0]['authorizationToken']
docker_login_command = f'docker login -u {username} -p {password} {registry}'

# Log in to the ECR registry
subprocess.call(docker_login_command, shell=True)

# Build your Docker image (replace 'path_to_Dockerfile' and 'tag' with your values)
docker_build_command = f'docker build -t {ecr_repository_name} -f path_to_Dockerfile .'
subprocess.call(docker_build_command, shell=True)

# Tag the Docker image for the ECR repository
docker_tag_command = f'docker tag {ecr_repository_name}:latest {registry}/{ecr_repository_name}:latest'
subprocess.call(docker_tag_command, shell=True)

# Push the Docker image to ECR
docker_push_command = f'docker push {registry}/{ecr_repository_name}:latest'
subprocess.call(docker_push_command, shell=True)

print(f'Docker image {ecr_repository_name} pushed to ECR repository {ecr_repository_name}')


# Customize for your use case
Replace the following placeholders with your specific values:

* 'your-aws-region': Replace with your AWS region (e.g., 'us-east-1').
* 'your-ecr-repo-name': Replace with the name of your ECR repository.
* 'path_to_Dockerfile': Replace with the path to your Dockerfile.
* 'tag': Replace with the desired tag for your Docker image.

# Prupose of this script does the following:

1. Initializes a Boto3 ECR client.
2. Retrieves an authentication token for Docker to log in to your ECR registry.
3. Logs in to the ECR registry using Docker.
4. Builds your Docker image.
5. Tags the Docker image with the ECR repository URI.
6. Pushes the Docker image to the specified ECR repository.


# Note:
Make sure you have the necessary AWS IAM permissions to perform these operations on your ECR repository.

## 2. Pulling Images:

Use the docker-py library to pull Docker images from an ECR repository in Python:

In [ ]:
import docker

# Initialize the Docker client
client = docker.from_env()

# Replace with your ECR image URL and credentials
image_url = 'your-account-id.dkr.ecr.your-region.amazonaws.com/your-repo:your-tag'
username = 'AWS'
password = '<your-ecr-password>'

# Log in to ECR
client.login(username=username, password=password, registry=image_url)

# Pull the image
client.images.pull(image_url)

print(f'Pulled image: {image_url}')

## 2. Logging In:

Logging in to ECR using Python and the AWS SDK (Boto3) is similar to the command-line example mentioned earlier:

In [ ]:
import boto3
import docker

# Initialize AWS and ECR clients
ecr_client = boto3.client('ecr', region_name='your-region')
docker_client = docker.from_env()

# Get the ECR login token
response = ecr_client.get_authorization_token()
token = response['authorizationData'][0]['authorizationToken']
username, password = base64.b64decode(token).decode().split(':')

# Log in to ECR
docker_client.login(username, password, registry='your-account-id.dkr.ecr.your-region.amazonaws.com')

print('Logged in to ECR')

## 3. Listing Repositories:

List ECR repositories using the AWS SDK (Boto3) in Python:

In [ ]:
import boto3

# Initialize the ECR client
ecr_client = boto3.client('ecr', region_name='your-region')

# List ECR repositories
response = ecr_client.describe_repositories()
repositories = response['repositories']

for repo in repositories:
    print(f"Repository Name: {repo['repositoryName']}")

## 4. Scanning Images:

Initiate image scans using the AWS SDK (Boto3) in Python:

In [ ]:
import boto3

# Initialize the ECR client
ecr_client = boto3.client('ecr', region_name='your-region')

# Specify the repository and image tag
repository_name = 'your-repo'
image_tag = 'your-tag'

# Start an image scan
response = ecr_client.start_image_scan(
    repositoryName=repository_name,
    imageId={'imageTag': image_tag}
)

print(f"Scan status: {response['imageScanStatus']['status']}")

## 5. Deleting Images:
**2 methods**
1. Directly via AWS CLI
2. Python scipt that automates the process

### 1. Directly via AWS CLI

**Step 1: List Images in the Repository**

Before deleting images, you need to list the images in the repository to identify the image digest or tag that you want to delete.

In [ ]:
aws ecr describe-images --repository-name <repository-name>

Replace <repository-name> with the name of your ECR repository.

**Step 2: Delete the Image**

Once you have identified the image tag or digest you want to delete, you can use the AWS CLI to delete it:

In [ ]:
aws ecr batch-check-layer-availability --region <region> --repository-name <repository-name> --image-ids imageTag=<tag>
aws ecr batch-delete-image --region <region> --repository-name <repository-name> --image-ids imageTag=<tag>

Replace <region> with your AWS region and <repository-name> and <tag> with the name of the ECR repository and the image tag or digest you want to delete.

Please note that batch-delete-image is a destructive operation, and it permanently deletes the specified images. Be cautious when using it.

### 2. Python scipt that automates the process

If you want to perform image deletion programmatically using Python, you can create a Python script that executes these AWS CLI commands using the subprocess module:

In [ ]:
import subprocess

# Define the ECR repository name and image tag to delete
repository_name = 'your-repo-name'
image_tag = 'your-image-tag'

# Step 1: Batch check layer availability
check_layer_command = f'aws ecr batch-check-layer-availability --region your-region --repository-name {repository_name} --image-ids imageTag={image_tag}'
subprocess.run(check_layer_command, shell=True, check=True)

# Step 2: Batch delete image
delete_image_command = f'aws ecr batch-delete-image --region your-region --repository-name {repository_name} --image-ids imageTag={image_tag}'
subprocess.run(delete_image_command, shell=True, check=True)

print(f"Deleted image {repository_name}:{image_tag}")


Replace 'your-region', 'your-repo-name', and 'your-image-tag' with your AWS region, ECR repository name, and the image tag you want to delete.

Remember to use this script with caution, as it permanently removes images from your ECR repository. Make sure you have the necessary AWS IAM permissions to perform this action.

## 6. Managing Lifecycle Policies

Managing lifecycle policies in Amazon Elastic Container Registry (ECR) allows you to automatically clean up old images based on certain rules or policies. Below are the steps and code for managing lifecycle policies in ECR:

**Step 1: Authenticate with AWS**

Ensure that you are authenticated with AWS using the AWS CLI. You should have the necessary permissions to manage repositories and policies in ECR.

**Step 2: Create a JSON Policy File**

You need to create a JSON file that defines your ECR lifecycle policy. The policy specifies the rules for image retention and cleanup. Here's an example JSON policy:

In [ ]:
{
  "rules": [
    {
      "rulePriority": 1,
      "description": "Keep 30 most recent images",
      "selection": {
        "tagStatus": "untagged",
        "countType": "imageCountMoreThan",
        "countNumber": 30
      },
      "action": {
        "type": "expire"
      }
    },
    {
      "rulePriority": 2,
      "description": "Keep images tagged with 'latest' for 7 days",
      "selection": {
        "tagStatus": "tagged",
        "tagPrefixList": ["latest"],
        "countType": "sinceImagePushed",
        "countNumber": 7
      },
      "action": {
        "type": "expire"
      }
    }
  ]
}


This JSON file defines two rules: one to keep the 30 most recent untagged images and another to keep images tagged with 'latest' for 7 days before expiring them.

**Step 3: Create or Update the Lifecycle Policy**

To create or update a lifecycle policy, you can use the put-lifecycle-policy AWS CLI command:

In [ ]:
aws ecr put-lifecycle-policy --repository-name <repository-name> --lifecycle-policy-text file://<path-to-policy-file>

Replace <repository-name> with the name of your ECR repository and <path-to-policy-file> with the path to the JSON policy file you created in Step 2.

**Step 4: Describe the Lifecycle Policy (Optional)**

You can use the describe-lifecycle-policy AWS CLI command to view the details of the lifecycle policy:

In [ ]:
aws ecr describe-lifecycle-policy --repository-name <repository-name>

This step is optional but can be useful for verifying the policy settings.

**Step 5: Apply the Policy to the Repository (Optional)**

By default, when you create or update a lifecycle policy, it does not automatically apply to existing images in the repository. To apply the policy to existing images, you can use the start-lifecycle-policy-preview AWS CLI command:

In [ ]:
aws ecr start-lifecycle-policy-preview --repository-name <repository-name>

This step is also optional, and you may choose to wait for new images to be subject to the policy.

The policy will automatically manage image retention and cleanup based on the rules defined in the JSON policy file.

## 7. Copying Images:  

Copying Docker images from one Elastic Container Registry (ECR) repository to another can be accomplished using the AWS Command Line Interface (CLI). Below are the steps and code to copy Docker images between ECR repositories:

**Step 1: Authenticate with AWS**

Ensure that you are authenticated with AWS using the AWS CLI. You should have the necessary permissions to access both the source and destination ECR repositories.

**Step 2: Check Layer Availability (Optional)**

Before copying an image, it's a good practice to check the layer availability in the destination repository. This step is optional but can be used to ensure that the necessary image layers are available in the destination repository.

In [ ]:
aws ecr batch-check-layer-availability --region <destination-region> --repository-name <destination-repo-name> --image-ids imageTag=<tag>

Replace <destination-region> with the AWS region of the destination repository, <destination-repo-name> with the name of the destination repository, and <tag> with the image tag you want to copy.

**Step 3: Authenticate with the Destination Repository**

To push the image to the destination repository, you need to authenticate with it.

In [ ]:
aws ecr get-login-password --region <destination-region> | docker login --username AWS --password-stdin <destination-account-id>.dkr.ecr.<destination-region>.amazonaws.com


Replace <destination-region> with the AWS region of the destination repository, <destination-account-id> with the AWS account ID of the destination repository.

**Step 4: Tag the Image**

Tag the Docker image from the source repository with the destination repository URI.

In [ ]:
docker tag <source-repo-uri>:<tag> <destination-repo-uri>:<tag>


Replace <source-repo-uri> with the URI of the source repository, <tag> with the image tag, and <destination-repo-uri> with the URI of the destination repository.

**Step 5: Push the Image to the Destination Repository**

Push the Docker image to the destination ECR repository.

In [ ]:
docker push <destination-repo-uri>:<tag>

Replace <destination-repo-uri> with the URI of the destination repository and <tag> with the image tag.

**Step 6: Verify the Image in the Destination Repository**

You can now verify that the Docker image has been copied to the destination repository by listing the images in the destination repository:

In [ ]:
aws ecr list-images --repository-name <destination-repo-name> --region <destination-region>

Replace <destination-repo-name> with the name of the destination repository and <destination-region> with the AWS region of the destination repository.

By following these steps and using the AWS CLI and Docker commands, you can copy Docker images from one ECR repository to another.

## 8. Integrating with CI/CD using GitLab CI/CD

Integrating Helm with your CI/CD pipeline for managing Docker-based applications in Kubernetes involves several steps. Helm is a package manager for Kubernetes that simplifies the deployment and management of applications. Below are the steps to integrate Helm with Docker in a CI/CD pipeline:

**Step 1: Prepare Your Helm Charts**

Make sure you have Helm charts for your applications ready in your project repository. Helm charts define how your application should be deployed in Kubernetes, including Docker image references, configuration values, and dependencies.

**Step 2: Create or Edit .gitlab-ci.yml**

Edit your project's .gitlab-ci.yml file to define the CI/CD pipeline stages for building and deploying Docker images and managing Helm charts. Here's a sample .gitlab-ci.yml file:

In [ ]:
stages:
  - build
  - deploy

variables:
  DOCKER_IMAGE_NAME: your-docker-image-name
  DOCKER_IMAGE_TAG: $CI_COMMIT_SHORT_SHA
  HELM_CHART_NAME: your-helm-chart-name

before_script:
  - # Authenticate with your container registry (e.g., Docker Hub)
    - echo "$DOCKER_PASSWORD" | docker login -u "$DOCKER_USERNAME" --password-stdin

build:
  stage: build
  script:
    - # Build your Docker image
      - docker build -t $DOCKER_IMAGE_NAME:$DOCKER_IMAGE_TAG .
    - # Push the Docker image to your container registry
      - docker push $DOCKER_IMAGE_NAME:$DOCKER_IMAGE_TAG
  only:
    - branches
  except:
    - tags

deploy:
  stage: deploy
  script:
    - # Deploy Helm chart to your Kubernetes cluster
      - helm upgrade --install $HELM_CHART_NAME ./path/to/helm/chart --set image.tag=$DOCKER_IMAGE_TAG
  only:
    - branches
  except:
    - tags


In this example:

Replace your-docker-image-name with the name of your Docker image.
Replace your-helm-chart-name with the name of your Helm chart.
The Docker image is built and pushed to your container registry (e.g., Docker Hub).
The Helm chart is deployed to your Kubernetes cluster using the Docker image tag as a variable.

**Step 3: Commit and Push .gitlab-ci.yml**

Commit the .gitlab-ci.yml file to your GitLab repository and push it to trigger the CI/CD pipeline.

**Step 4: CI/CD Pipeline Execution**

GitLab CI/CD will automatically build your Docker image and push it to your container registry when changes are pushed to the repository. After a successful build, the Helm chart is deployed to your Kubernetes cluster with the updated Docker image tag.

Ensure that you have Docker, Helm, and kubectl installed in your CI/CD runner environment. Additionally, configure environment variables for your container registry credentials (e.g., DOCKER_USERNAME and DOCKER_PASSWORD) securely in your GitLab CI/CD settings.

This configuration allows you to automate the deployment of your Docker-based applications in Kubernetes using Helm as part of your CI/CD pipeline. Make any necessary adjustments based on your specific project requirements and container registry configuration.